In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
startPath = "/Users/addisonwu/aw2418@princeton.edu - Google Drive/My Drive/Clear Air Turbulence Localization Deep Learning/infraGA/raw_raypath_datasets"
endPath = "/Users/addisonwu/aw2418@princeton.edu - Google Drive/My Drive/Clear Air Turbulence Localization Deep Learning/infraGA/scraped_raypath_datasets"

In [ ]:
for filename in os.listdir(startPath):
    #Set the altitude of the microphone
    microphone_alt = np.around(np.random.uniform(3, 20), decimals=2)
    microphone_r = np.around(np.random.uniform(10, 900), decimals=3)

    #Read datafile and ensure it is float file
    df = pd.read_table(os.path.join(startPath, filename), dtype=float)
    df = df.astype(float)

    #parse for relevant data, r and z must be within the microphone's pickup range
    #Spherical distribution? Pythagorean inequality
    drop_row_list = []
    for i in range(0, len(df)):
        if (microphone_r - float(df["# r [km]"][i]))**2 + (microphone_alt - float(df["z [km]"][i]))**2 > 2.5**2:  #microphone with capture radius of 2.5 km
            drop_row_list.append(i) # data is out of range of microphone discard it
    df = df.drop(labels = drop_row_list, axis = 0) #discard said rows

    #now we need to adjust the time column so that it's scaled down (we can only use what we know)
    #the times are set relative to when the microphone registers, not how long it takes after emitted from source
    #drop irrelevant columns
    df["time [s]"] = df["time [s]"] - min(df["time [s]"])
    df = df.drop(columns=["r [km]", "z [km]", "trans. coeff. [dB]"])

    #write and save to a new file name
    newfile = "MicAlt_" + str(microphone_alt) + "_MicDistance_" + str(microphone_r) + "_" + filename
    path = "/Users/addisonwu/aw2418@princeton.edu - Google Drive/My Drive/Clear Air Turbulence Localization Deep Learning/infraGA/scraped_raypath_datasets"
    with open(os.path.join(path, newfile), 'w') as file:
      file.write(df)

    newfile = f"MicAlt_{microphone_alt}_MicDistance_{microphone_r}_{filename}"
    output_file_path = os.path.join("/Users/addisonwu/aw2418@princeton.edu - Google Drive/My Drive/Clear Air Turbulence Localization Deep Learning/infraGA/scraped_raypath_datasets", newfile)
    df.to_csv(output_file_path, sep='\t', index=False, header=False)